In [15]:
# Do necessary Imports
import os
import numpy as np
import pandas as pd
from pathlib import Path
import quandl

In [16]:
# Read BSE 500 Constituents csv file
bse500_metadata = pd.read_csv('BSE 500index_Constituents.csv')
bse500_metadata.head(2)

,Scrip Code,COMPANY,ISIN No.,Close Price
0,523395,3M India Ltd,INE470A01017,18033.6
1,541988,AAVAS Financiers Ltd,INE216P01012,1012.4


In [17]:
# Get all 500+ tickers
tickers = list(bse500_metadata['Scrip Code'])
print('Number of tickers: ',len(tickers))

Number of tickers:  501


In [18]:
# tickers = tickers[:10]

In [19]:
quandl.ApiConfig.api_key = 'yoK3TWKCEXy1QsxwyNf6'

In [20]:
# Start Bulk download in a loop and create a Dataframe

def get(tickers): 
    def data(ticker):
        print('Processing...', f'BSE/BOM{ticker}')
        return (quandl.get(f'BSE/BOM{ticker}'))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['ticker', 'date']))
               
df = get(tickers)

Processing... BSE/BOM523395
Processing... BSE/BOM541988
Processing... BSE/BOM500488
Processing... BSE/BOM500410
Processing... BSE/BOM512599
Processing... BSE/BOM542066
Processing... BSE/BOM541450
Processing... BSE/BOM532921
Processing... BSE/BOM533096
Processing... BSE/BOM539254
Processing... BSE/BOM540691
Processing... BSE/BOM535755
Processing... BSE/BOM540025
Processing... BSE/BOM500003
Processing... BSE/BOM532683
Processing... BSE/BOM532331
Processing... BSE/BOM500710
Processing... BSE/BOM533573
Processing... BSE/BOM539523
Processing... BSE/BOM506767
Processing... BSE/BOM532749
Processing... BSE/BOM500008
Processing... BSE/BOM540902
Processing... BSE/BOM500425
Processing... BSE/BOM533758
Processing... BSE/BOM508869
Processing... BSE/BOM500877
Processing... BSE/BOM542484
Processing... BSE/BOM500477
Processing... BSE/BOM533271
Processing... BSE/BOM500820
Processing... BSE/BOM540975
Processing... BSE/BOM532830
Processing... BSE/BOM506820
Processing... BSE/BOM500027
Processing... BSE/BO

Processing... BSE/BOM539957
Processing... BSE/BOM500266
Processing... BSE/BOM500265
Processing... BSE/BOM532720
Processing... BSE/BOM500520
Processing... BSE/BOM532756
Processing... BSE/BOM533088
Processing... BSE/BOM532313
Processing... BSE/BOM540768
Processing... BSE/BOM531213
Processing... BSE/BOM500109
Processing... BSE/BOM531642
Processing... BSE/BOM532500
Processing... BSE/BOM540749
Processing... BSE/BOM500271
Processing... BSE/BOM539981
Processing... BSE/BOM532865
Processing... BSE/BOM542650
Processing... BSE/BOM538962
Processing... BSE/BOM532539
Processing... BSE/BOM532819
Processing... BSE/BOM541195
Processing... BSE/BOM513377
Processing... BSE/BOM533286
Processing... BSE/BOM517334
Processing... BSE/BOM532892
Processing... BSE/BOM526299
Processing... BSE/BOM500290
Processing... BSE/BOM534091
Processing... BSE/BOM533398
Processing... BSE/BOM539551
Processing... BSE/BOM524816
Processing... BSE/BOM532234
Processing... BSE/BOM513023
Processing... BSE/BOM532504
Processing... BSE/BO

In [21]:
# Verify the head of Dataframe
df.head(2)

Open  High   Low  Close  WAP  No. of Shares  No. of Trades  \
ticker date                                                                     
523395 1991-06-26  80.0  85.0  80.0   85.0  0.0            0.0            0.0   
       1991-06-27  77.5  82.5  77.5   80.0  0.0            0.0            0.0   

                   Total Turnover  Deliverable Quantity  \
ticker date                                               
523395 1991-06-26             0.0                   NaN   
       1991-06-27             0.0                   NaN   

                   % Deli. Qty to Traded Qty  Spread H-L  Spread C-O  
ticker date                                                           
523395 1991-06-26                        NaN         5.0         5.0  
       1991-06-27                        NaN         5.0         2.5

In [22]:
# Create a data store to save the results
DATA_STORE = Path('bse500.h5')

In [23]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('/quandl/bse/bse500', df)

In [24]:
# with pd.HDFStore(DATA_STORE) as store:
#     print(store.info())

### Create raw CSV files for Ingestion for each ticker

In [25]:
# filter required columns from Dataframe
columns = ['Open','High','Low','Close','No. of Trades']

In [26]:
prices = df[columns]
prices.head(2)

Open  High   Low  Close  No. of Trades
ticker date                                              
523395 1991-06-26  80.0  85.0  80.0   85.0            0.0
       1991-06-27  77.5  82.5  77.5   80.0            0.0

In [27]:
# Rename the columns as per zipline requirement
prices.columns = ['open','high','low','close','volume']
prices.head(2)

open  high   low  close  volume
ticker date                                       
523395 1991-06-26  80.0  85.0  80.0   85.0     0.0
       1991-06-27  77.5  82.5  77.5   80.0     0.0

In [28]:
# Write a csv file for each ticker
for ticker in tickers:
    df = prices.loc[ticker]
    df.to_csv(f'bse500/BOM{ticker}.csv', index=True)